# Chapter 9: Credit Risk and Credit Default Swaps


This chapter focuses on credit risk, exploring how to measure credit risk exposure and manage it using credit default swaps (CDSs). The first section defines credit risk and introduces the concept of expected loss (EL) along with the three core elements of credit risk. The second section examines various credit risk models. In the third section, we discuss methods for estimating credit spreads. The final section focuses on credit risk management, beginning with an introduction to CDS and its role in hedging credit risk, followed by a discussion of its drawbacks and limitations.

Chapter Objectives:

1. Understand credit risk and expected loss
2. Learn different credit risk models
3. Estimate credit spreads of debt instruments
4. Comprehend how credit default swaps work and the basic pricing mechanisms 

In [1]:
# First method
promised_pmt = 1000
expected_pmt = 1000*0.8+500*0.2
# EL is the difference between promosed and expected payments
EL = promised_pmt - expected_pmt
print("the expected loss is", EL)

the expected loss is 100.0


In [2]:
# Second method
EAD = 1000
PD = 0.2
LGD = 0.5
print(f"EL = EAD*PD*LGD = {EAD}*{PD}*{LGD} =",EAD*PD*LGD)

EL = EAD*PD*LGD = 1000*0.2*0.5 = 100.0


In [3]:
EAD = 300000 * 80
PD = 0.2
LGD = 1-0.45
print(f"EL = EAD*PD*LGD = {EAD}*{PD}*{LGD} =",EAD*PD*LGD)


EL = EAD*PD*LGD = 24000000*0.2*0.55 = 2640000.0


In [4]:
#!pip install --user lifelines
import pandas as pd 
from lifelines import CoxPHFitter  
# Define the dataset 
data = {     
    'Loan_Amount': [10000, 20000, 15000, 30000, 25000, 12000, 
                    8000, 5000, 22000, 27000],     
    'Interest_Rate': [5.5, 4.0, 6.0, 3.5, 4.5, 5.0, 4.0, 6.5, 4.5, 3.0],     
    'Loan_Term': [36, 60, 48, 24, 36, 60, 24, 36, 48, 60],     
    'Age': [30, 40, 35, 50, 45, 25, 55, 60, 32, 38],     
    'Income': [60000, 85000, 72000, 90000, 62000, 48000, 
               78000, 95000, 67000, 52000],     
    'Defaulted': [1, 0, 0, 1, 0, 0, 1, 0, 1, 0],     
    'Time': [12, 60, 24, 36, 48, 30, 18, 40, 15, 55] }  
df = pd.DataFrame(data)  
# Instantiate and fit the Cox Proportional Hazards model 
cph = CoxPHFitter() 
cph.fit(df, duration_col='Time', event_col='Defaulted')  
# Print the summary 
cph.print_summary()
#print(cph.baseline_hazard_)


C:\Users\hlliu2\AppData\Roaming\Python\Python39\site-packages\lifelines\fitters\coxph_fitter.py:1614: ConvergenceWarning: Newton-Raphson failed to converge sufficiently. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
  warnings.warn(


<lifelines.CoxPHFitter: fitted with 10 total observations, 6 right-censored observations>
             duration col = 'Time'
                event col = 'Defaulted'
      baseline estimation = breslow
   number of observations = 10
number of events observed = 4
   partial log-likelihood = -2.00
         time fit was run = 2024-08-12 10:14:18 UTC

---
               coef exp(coef)  se(coef)  coef lower 95%  coef upper 95% exp(coef) lower 95% exp(coef) upper 95%
covariate                                                                                                      
Loan_Amount   -0.00      1.00      0.00           -0.00            0.00                1.00                1.00
Interest_Rate -2.61      0.07      4.17          -10.79            5.56                0.00              259.19
Loan_Term     -0.34      0.71      0.35           -1.03            0.35                0.36                1.42
Age           -0.74      0.48      1.03           -2.75            1.27                0.06                3.57
Income         0.00      1.00      0.00           -0.00            0.00                1.00                1.00

               cmp to     z    p  -log2(p)
covariate                                 
Loan_Amount      0.00 -0.66 0.51      0.97
Interest_Rate    0.00 -0.63 0.53      0.91
Loan_Term        0.00 -0.96 0.34      1.58
Age              0.00 -0.72 0.47      1.08
Income           0.00  0.46 0.65      0.62
---
Concordance = 0.96
Partial AIC = 14.00
log-likelihood ratio test = 12.38 on 5 df
-log2(p) of ll-ratio test = 5.06

In [5]:
from scipy.stats import norm
from numpy import exp, log, sqrt
import numpy as np
from scipy.optimize import minimize

S0 = 4
std_s = 0.70
r = 0.06
T = 1
D = 10

def d1(s,r,t,x,std):
    return (log(s/x)+(r+0.5*std**2)*t)/(std*sqrt(t))

def d2(s,r,t,x,std):
    return (log(s/x)+(r-0.5*std**2)*t)/(std*sqrt(t))
   
def equations(z):
    std_v = z[0]
    V0 = z[1]

    f = np.zeros(2)
    f[0] = S0-V0*norm.cdf(d1(V0,r,T,D,std_v))+D*exp(-r*T)*norm.cdf(d2(V0,r,T,D,std_v))
    f[1] = std_s*S0-norm.cdf(d1(V0,r,T,D,std_v))*std_v*V0
    return np.dot(f,f)

def cons_x(z):
    std_v = z[0]
    V0 = z[1]
   
    f = np.zeros(3)
    f[0] = V0-S0
    f[1] = S0+D-V0
    f[2] = std_v
    return f

cons = {'type' : 'ineq', 'fun': cons_x}
res = minimize(equations, (std_s, S0), method='SLSQP', constraints=cons)

print(res)

std_v=res.x[0]
V0=res.x[1]
print("std_v is", std_v)
print("V0 is", V0)
print("d2 is", d2(V0,r,T,D,std_v))
print("prob of default is", norm.cdf(-d2(V0,r,T,D,std_v)))


 message: Optimization terminated successfully
 success: True
  status: 0
     fun: 8.275467861531818e-08
       x: [ 2.193e-01  1.336e+01]
     nit: 15
     jac: [ 3.777e-04 -5.116e-04]
    nfev: 52
    njev: 15
std_v is 0.2192732199097569
V0 is 13.359725126427522
d2 is 1.4849926871271162
prob of default is 0.06877288064769242


In [6]:
import numpy_financial as npf
# Calculate expected cash flow and discount rate
expected_pmt = 1000*0.8+500*0.2
rf = 0.05
# Calculate bond price
P_unsystematic = expected_pmt/(1+rf)
print(f"If the credit risk is unsystematic, the bond price is", round(P_unsystematic,3))
# Calculate yield to maturity
ytm_unsystematic = npf.irr([-P_unsystematic, 1000])
print(f"Yield to maturity of the bond price is", round(ytm_unsystematic,3))
# Calculate credit spread
credit_spread_unsystematic = ytm_unsystematic - rf
print(f"credit spread is", round(credit_spread_unsystematic,3))


If the credit risk is unsystematic, the bond price is 857.143
Yield to maturity of the bond price is 0.167
credit spread is 0.117


In [7]:
# Obtain the risk premium
risk_premium = 0.05
# Calculate bond price
P_systematic = expected_pmt/(1+rf+risk_premium)
print(f"If the credit risk is systematic, the bond price is", P_systematic)
# Calculate yield to maturity
ymt_systematic = npf.irr([-P_systematic, 1000])
print(f"Yield to maturity of the bond price is", ymt_systematic)
# Calculate credit spread
credit_spread_systematic = ymt_systematic - rf
print(f"credit spread is", credit_spread_systematic)


If the credit risk is systematic, the bond price is 818.1818181818181
Yield to maturity of the bond price is 0.22222222222222232
credit spread is 0.17222222222222233


In [8]:
# First calculate the risk-free bond price
P_rf = 1000/(1+rf)
# The CDS price if credit risk is unsystematic
CDS_unsystematic = P_rf - P_unsystematic
print(f"If the credit risk is unsystematic, CDS price is", CDS_unsystematic)
# The CDS price if credit risk is systematic
CDS_systematic = P_rf - P_systematic
print(f"If the credit risk is systematic, CDS price is", CDS_systematic)


If the credit risk is unsystematic, CDS price is 95.23809523809518
If the credit risk is systematic, CDS price is 134.19913419913416


In [9]:
# Get current assets and current liabilities from you
CA = float(input("What's current assets (in $m)?\n"))
CL = float(input("What's current liabilities (in $m)?\n"))
# Get total assets and retained earnings from you
TA = float(input("What's total assets (in $m)?\n"))
RR = float(input("What's retained earnings (in $m)?\n"))
# Get EBIT and market capitalization from you
EBIT = float(input("What's EBIT (in $m)?\n"))
mkt_cap = float(input("What's market capitalization (in $m)?\n"))
# Get total liabilities and sales from you
TL = float(input("What's total liabilities (in $m)?\n"))
SALES = float(input("What's total sales (in $m)?\n"))

# Calculate A, B, C, D, and E
A = (CA-CL)/TA
B = RR/TA
C = EBIT/TA
D = mkt_cap/TL
E = SALES/TA

# Calcualte the Z-score and print out
Z = 1.2*A + 1.4*B + 3.3*C + 0.6*D + E
print("the Z-score is", Z)

# Predict default based on Z-score
if Z>3.0:
    print("the company is unlikely to default in the next year")
elif Z>2.7:
    print("the company is on alert")
elif Z>1.8:
    print("the company is likely to default in the next year")
else:
    print("the company is highly likely to default in the next year")

What's current assets (in $m)?
26717
What's current liabilities (in $m)?
14248
What's total assets (in $m)?
52148
What's retained earnings (in $m)?
-5399
What's EBIT (in $m)?
1994
What's market capitalization (in $m)?
593871
What's total liabilities (in $m)?
28418
What's total sales (in $m)?
31536
the Z-score is 13.411531285278544
the company is unlikely to default in the next year
